# **Import**
---

In [1]:
import numpy as np 
import os
import shutil
import math
import tensorflow as tf
import json 

from time import time
from tensorflow import keras
from keras.preprocessing import image
from keras.models import Model, load_model
from keras import backend as K
from keras.applications.vgg16 import VGG16, preprocess_input #224*224
from keras.applications.xception import Xception, preprocess_input, decode_predictions #299*299
from keras.applications.mobilenet import MobileNet, preprocess_input, decode_predictions #224*224
from tqdm import tqdm
from keras_preprocessing.image import load_img
from keras_preprocessing.image import img_to_array
print("Tensorflow version: "+tf.__version__)
print("Keras version: " + tf.keras.__version__)

Tensorflow version: 2.9.2
Keras version: 2.9.0


# **Database download**
---

In [ ]:
!wget https://github.com/sidimahmoudi/facenet_tf2/releases/download/AI_MIR_CLOUD/MIR_DATASETS_B.zip 
!unzip -q /content/MIR_DATASETS_B.zip
!rm MIR_DATASETS_B.zip
!rm -f sample_data

# **Datbase preprocess**
---

In [3]:
for animal in os.listdir('MIR_DATASETS_B'):
  for type_animal in os.listdir(os.path.join('MIR_DATASETS_B', animal)):
    for file in os.listdir(os.path.join('MIR_DATASETS_B', animal, type_animal)):
      shutil.copy2(os.path.join('MIR_DATASETS_B', animal, type_animal, file), os.path.join('MIR_DATASETS_B'))

# **Indexation in JSON**
---

In [18]:
def dl_predict(filename, model, size):
    img = load_img(filename, target_size = size)
    img = img_to_array(img)
    img = img.reshape((1, img.shape[0], img.shape[1], img.shape[2]))
    img = preprocess_input(img)
    feature = model.predict(img)
    feature = np.array(feature[0])
    return feature

def generate(generate_func, source, output):
    start = time()
    feature = generate_func(source)
    save(feature, output)
    del feature
    print(f'[INFO] Indexation : {generate_func.__name__} --> Done')
    return round(time() - start, 3)

def save(data, file_name):
    with open(file_name, 'w') as f:
        json.dump(data, f, cls = JsonCustomEncoder)

class JsonCustomEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, (np.ndarray, np.number)):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

In [17]:
def VGG16_init():
    # https://keras.io/api/applications/vgg/
    return VGG16(include_top = False, weights ='imagenet', input_shape = (224, 224, 3), pooling = 'avg')

def Xception_init():
    # https://keras.io/api/applications/xception/
    return Xception(include_top = False, weights ='imagenet', input_shape = (299, 299, 3), pooling = 'avg')

def MobileNet_init():
    # https://keras.io/api/applications/mobilenet/
    return MobileNet(include_top = True, weights ='imagenet', input_shape = (224, 224, 3), pooling = 'avg')

def generateVGG16(folder):
    model = VGG16_init()
    data = list()
    for path in tqdm(os.listdir(folder)):
      if '.jpg' in path:
        feature = dl_predict(os.path.join(folder, path), model, (224, 224))
        num_image, _ = path.split(".")
        data.append({num_image : feature})
    return data

def generateXception(folder):
    model = Xception_init()
    data = list()
    for path in tqdm(os.listdir(folder)):
      if '.jpg' in path:
        feature = dl_predict(os.path.join(folder, path), model, (299, 299))
        num_image, _ = path.split(".")
        data.append({num_image : feature})
    return data

def generateMobileNet(folder):
    model = MobileNet_init()
    data = list()
    for path in tqdm(os.listdir(folder)):
      if '.jpg' in path:
        feature = dl_predict(os.path.join(folder, path), model, (224, 224))
        num_image, _ = path.split(".")
        data.append({num_image : feature})
    return data
    
  

In [ ]:
time_VGG16 = generate(generateVGG16, 'MIR_DATASETS_B/', 'VGG16.json')

In [ ]:
time_Xcpetion = generate(generateXception, 'MIR_DATASETS_B/', 'XCEPTION.json')

In [ ]:
time_MobileNet = generate(generateMobileNet, 'MIR_DATASETS_B/', 'MOBILENET.json')